# Анализ базы данных книжного онлайн-сервиса.


Наша компания купила крупный сервис для чтения книг по подписке. Наша  задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.
В ходе работы мы:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определим автора с самой высокой средней оценкой книг;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

Описание данных:

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.


###  Загрузим данные и посмотри на данные.
####  Импортируем библиотеки. 

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine

#### Подключимся к базе данных.

In [2]:
# устанавливаем параметры
db_config = {'user': '-censored-', # имя пользователя
'pwd': '-censored-', # пароль
'host': '-censored-',
'port': '-censored-', # порт подключения
'db': '-censored-'} # название базы данных
connection_string = ''-censored-''.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


#### Выгрузим первые строки таблицы `books`

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


#### Выгрузим первые строки таблицы `authors`

In [4]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Выгрузим первые строки таблицы `publishers`

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


#### Выгрузим первые строки таблицы `ratings`

In [6]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


#### Выгрузим первые строки таблицы `reviews`

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


В данных представлена информация о 1000 книгах, 6 авторах, которые их написали. В таблице publishers информация о 340 изданиях, где были выпущены книги. В таблийе rating содержится информация о 6456 рейтингах и пользователях, которые оставили оценки. Так же в таблице reviews информация о 2793 обзорах на книги.

### Анализ данных.

#### Посчитаем, сколько книг вышло после 1 января 2000 года.

In [8]:
query = '''
           SELECT COUNT(book_id) 
           FROM books 
           WHERE publication_date > '2000-01-01'
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг

#### Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [9]:
query = '''
           SELECT b.book_id AS book_id,
                  title,
                  COUNT(DISTINCT review_id) AS count_review,
                  AVG(rating) AS avg_rating
           FROM books AS b
           LEFT JOIN reviews AS re ON b.book_id=re.book_id
           LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
           GROUP BY b.book_id
           
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


По количеству обзоров лидирует книга Twilight (Twilight 1).

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц( так мы исключим из анализа брошюры).

In [10]:
query = '''
           SELECT publisher,
                  COUNT(book_id) as count_books
           FROM books AS b 
           LEFT JOIN publishers AS pub ON b.publisher_id=pub.publisher_id
           WHERE num_pages > 50
           GROUP BY pub.publisher_id
           ORDER BY count_books DESC
           LIMIT 1
        '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count_books
0,Penguin Books,42


Наибольшее количество книг выпустило издательство Penguin Books, их количество равно 42 шт.

#### Определим автора с самой высокой средней оценкой книг
Будем учитывать только книги с 50 и более оценками.

In [11]:
query = '''
           WITH
           a AS (SELECT a.author as author,
                 b.book_id,
                        AVG(rating) OVER (PARTITION BY b.book_id) as avg_rating,
                        COUNT(rating_id) OVER (PARTITION BY b.book_id) as count_rating
                 FROM books AS b 
                 LEFT JOIN ratings AS ra ON b.book_id=ra.book_id
                 LEFT JOIN authors AS a ON b.author_id=a.author_id)
            
      
          SELECT author,
                 AVG(avg_rating) AS rating_avg
          FROM a
          WHERE count_rating > 50 
          GROUP BY author
          ORDER BY rating_avg DESC
          LIMIT 1 
       '''

con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автором набравшим самый высоких средний рейтинг является J.K. Rowling/Mary GrandPré, его рейтинг равен 4,28.

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''        
           WITH
           a AS (SELECT username,
                        COUNT(rating_id) AS count_rating
                 FROM ratings
                 GROUP BY username
                 HAVING COUNT(rating_id) > 48),
           b as (SELECT username,
                        COUNT(review_id) AS count_review
                 FROM reviews
                 GROUP BY username)
           SELECT AVG(count_review)
           FROM a
           LEFT JOIN b ON a.username=b.username
        '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


В среднем активные пользователи пишут по 24 обзора на книги.

### Вывод.

1. В данных представлена информация о 1000 книгах, 6 авторах, которые их написали. В таблице publishers информация о 340 изданиях, где были выпущены книги. В таблийе rating содержится информация о 6456 рейтингах и пользователях, которые оставили оценки. Так же в таблице reviews информация о 2793 обзорах на книги.
2. После 1 января 2000 года вышло 819 книг
3. По количеству обзоров лидирует книга Twilight (Twilight 1).
4. Наибольшее количество книг выпустило издательство Penguin Books, их количество равно 42 шт.
5. Автором набравшим самый высоких средний рейтинг является J.K. Rowling/Mary GrandPré, его рейтинг равен 4,28.
6. В среднем активные пользователи пишут по 24 обзора на книги.
